# Add new person to database

Take pictures of people which are supposed to have access to the system, and save them in folder

In [1]:
import cv2
import os
import cvlib

# Number of photos to take of each person
# More photos = more reliable embedding vector estimation
NUM_OF_PHOTOS = 50

# Path to folder in which photos are stored
IMAGES_PATH = 'face_images'

# Create directory for storing face images
# if it does not exist yet
try:
    os.mkdir(IMAGES_PATH)
except FileExistsError:
    pass


def take_pictures(persons_name):
    """
    Captures face photos by using camera
    and stores them in folder
    
    Arguments
    ---------
    pesons_name - string containing name of the
                  person pictures will be taken of
    """
    try:
        # Create a subdirectory to store photos for each person
        dir_name = os.path.join(IMAGES_PATH, persons_name)
        try:
            os.mkdir(dir_name)
        except FileExistsError:
            # This means we already have photos of that person
            print('{} already exists in database!'.format(persons_name))
            return
        
        # Instanciate capture object in order to
        # gain access to camera
        cap = cv2.VideoCapture(0)

        # Some aux flags used to count photos taken and start
        # of the process
        start = False
        photo_count = 0

        # Take NUM_OF_PHOTOS shots
        while photo_count != NUM_OF_PHOTOS:
            # Capture a frame from camera
            ret, frame = cap.read()
            # If reading was successful, continue
            # processing. If not, try again
            if not ret:
                print('Problem connecting to camera!')
                continue

            # Detect faces in the photo using cvlib
            # Algorithm will detect face closest to
            # camera with highest probability, but it
            # might also detect other faces around it, 
            # but in face recognition use case only one 
            # face is expected in front of camera anyway
            faces, _ = cvlib.detect_face(frame)

            # If at least one face was detected
            if len(faces) != 0:
                # Mark coordinates of the rectangle surrounding face
                (startX, startY) = faces[0][0], faces[0][1]
                (endX, endY) = faces[0][2], faces[0][3]

                # Draw white rectangle over face
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0,255,0), 2)

                # If 's' was pressed, the capturing has started
                if start:
                    # Extract only region surrounded by box
                    roi = frame[startY+2:endY-2, startX+2:endX-2]
                    # resize it to (160,160,3), as expected by NN model
                    resized = cv2.resize(roi, (160,160), interpolation = cv2.INTER_AREA)
                    # and save it to disc
                    save_path = os.path.join(dir_name, '{}.jpg'.format(photo_count + 1))
                    cv2.imwrite(save_path, resized)
                    photo_count += 1

            # Put some indicator text above image and display what camera sees
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, "{}, please look straight in the camera".format(persons_name),
                        (5, 30), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, "Collecting {}. photo".format(photo_count),
                        (5, 50), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            # Display what is currently captured by camera along with above text
            cv2.imshow("Collecting images of {}".format(persons_name), frame)

            # Listen for key presses
            k = cv2.waitKey(1)
            if k == ord('s'):
                start = True

            if k == ord('q'):
                break

        # Print output text indicating that photo shooting is finished
        print("\n{} image(s) saved to {}".format(photo_count, dir_name))

        # Release the camera
        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(e)
        cap.release()
        cv2.destroyAllWindows()
        return

Add new person to database

In [2]:
#name = 'Kristijan'

#take_pictures(name)

# Make face embeddings out of pictures

When images for people we want to have access to the
system are collected, we proceed to make embedding
vectors of them, average them and estimate the largest
Euclidian distance up to which we can consider faces
to belong to the same person

In [3]:
from keras.models import load_model
import os
import cv2
import numpy as np
from sklearn.preprocessing import normalize

# Path to folder where photos were stored
IMAGES_PATH = 'face_images'

# Path to folder where npy files are stored
# for each person, containing their averaged embedding
# vector and maximum deviation from that vector
EMBEDDINGS_PATH = 'embeddings'

# Create directory for storing embeddings
# if it does not exist yet
try:
    os.mkdir(EMBEDDINGS_PATH)
except FileExistsError:
    pass

# Load the model
model = load_model('pretrained_models/facenet_keras.h5')

# Summarize input and output shape
print(model.inputs)
print(model.outputs)

def create_embeddings(persons_name):
    """
    Images of person are passed through pretrained NN
    to produce 128 dimensional embedding vector for 
    each image, which are then averaged to obtain one
    vector to represent that person, which is then
    stored in .npy file
    
    Arguments
    ---------
    persons_name - string containing name of the person
    """
    emb_file = EMBEDDINGS_PATH + '/' + persons_name + '.npy'
    if os.path.exists(emb_file):
        print('Embeddings for {} already exist!'.format(persons_name))
        return
    
    path_to_photos = IMAGES_PATH + '/' + persons_name
    images = os.listdir(path_to_photos)
    embeddings = []
    
    for photo in images:
        face = cv2.imread(path_to_photos + '/' + photo).astype('float32')
        # standardize pixel values across channels (global)
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        # transform face into one sample
        sample = np.expand_dims(face, axis=0)
        
        # Form a 128 dimensional vector representing image
        # by passing image through the model
        embedding = model.predict(sample)[0]
        # and store it for person
        embeddings.append(embedding)
        
    embeddings = np.asarray(embeddings) # shape is (num_of_photos, 128)
    # Average all embedding vectors to obtain one vector
    # representing person and normalize it
    mean = normalize(np.mean(embeddings, axis=0).reshape(1,128), axis=1)
    
    emb_dict = {
        'name' : persons_name,
        'emb_vec' : mean
    }
    np.save(emb_file, emb_dict)
    
    print('Embeddings for {} created!'.format(persons_name))

Using TensorFlow backend.


[<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/cond/Identity:0' shape=(None, 128) dtype=float32>]


C:\Users\Kris\anaconda3\envs\tf_keras\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Create embeddings for all familiar people

In [4]:
#familiar_people = os.listdir(IMAGES_PATH)

#for person in familiar_people:
#    create_embeddings(person)

# Accessing the system

Situation: someone is trying to gain access to the system. We check if their
embedding matches any of familiar ones, and based on that allow them or deny
access.

In [5]:
import cv2
import os
import cvlib
import numpy as np
import time
from sklearn.preprocessing import normalize
from keras.models import load_model

# Load the model
model = load_model('pretrained_models/facenet_keras.h5')

# Path to folder where npy files are stored
# for each person, containing their averaged embedding
# vector and maximum deviation from that vector
EMBEDDINGS_PATH = 'embeddings'

# Recognition timeout in secs
TIMEOUT = 5

# Similarity Euclidian distance. If embedding vectors
# are farther away that this, face is considered to be
# of different person
SIMILARITY_THRESH = 0.7

# Font used for opencv
FONT = cv2.FONT_HERSHEY_SIMPLEX

def detect_face(frame):
    """
    Detect faces using cvlib, if there are any in the frame.
    
    Arguments
    ---------
    frame - frame captured by openCV
    
    Returns
    -------
    boolean showning whether a face was detected,
    isolated pixels containing only face if one was detected,
    list containing two opposite corners of rectangle containing
    face
    """
    # Detect face in the frame
    faces, _ = cvlib.detect_face(frame)
    
    # If at least one face was detected
    if len(faces) != 0:
        # Mark coordinates of the rectangle surrounding face
        (startX, startY) = faces[0][0], faces[0][1]
        (endX, endY) = faces[0][2], faces[0][3]

        # Extract only region surrounded by box
        roi = frame[startY+2:endY-2, startX+2:endX-2]
        # resize it to (160,160,3), as expected by NN model
        face = cv2.resize(roi, (160,160), interpolation = cv2.INTER_AREA).astype('float32')
        
        # standardize pixel values across channels 
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
                
        return True, face, [startX, startY, endX, endY]
    
    else:
        return False, None, None

    
def recognize_face(face, known_emb):
    """
    Compares embedding vector of a face passed as
    argument to vectors of people to which access
    is allowed, stored previously on disc
    
    Arguments
    ---------
    face - (160,160,3) shaped ndarray containing only a face
            which is to be compared to stored faces
                
    Returns
    -------
    boolean showing whether passed face was recognized,
    name of the person, if face was recongized
    """
    # Form a normalized 128 dimensional vector representing image
    # by passing image through the model
    embedding = normalize(model.predict(np.expand_dims(face, axis=0))[0].reshape(1,128), axis=1)
            
    for person in known_emb:
        emb = person['emb_vec']
        # Find L2 distance between newcomers face and a familiar one
        diff = np.linalg.norm(emb - embedding, axis=1)
        print('Distance: ', diff)
        # Empirical threshold large enough to distinct faces
        if diff < SIMILARITY_THRESH:
            return True, person['name']
        
    return False, None


def display_frame(orig_frame, text, has_rect=False, rect=None, rect_color=(255,255,255)):
    """
    Boilerplate code to display decorated frame with or without text and rectangle
    """
    frame = np.copy(orig_frame)
    cv2.putText(frame, text, (5, 30), FONT, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    if has_rect:
        # Draw green rectangle over face
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), rect_color, 2)
        
    # Display what is currently captured by camera along with above text
    cv2.imshow("Camera", frame)
    
    
def access():
    """
    Function requesting access to the system.
    Camera is turned on and faces are detected.
    If no face was detected, after a 5 sec timeout,
    system will go back to sleep.
    If face was detected, function proceeds to try and
    recognize it. If the face was recognized, it allows
    access to the system. Otherwise, it denies it.
    
    Returns
    -------
    boolean - True if acces was granted, False otherwise
    """
    # Wrapped up in try-except block to properly release
    # camera if something goes wrong
    try:
        # Mark the starting time, to be able to timeout
        # requests to access the system
        start = time.time()
        
        # Create list of known people's embeddings
        known_emb = []
        for file in os.listdir(EMBEDDINGS_PATH):
            emb = np.load(EMBEDDINGS_PATH + '/' + file, allow_pickle=True).item()
            known_emb.append(emb)
        
        # Instanciate capture object in order to
        # gain access to camera
        cap = cv2.VideoCapture(0)

        # Main loop looking for faces
        while True:
            # Capture a frame from camera
            ret, frame = cap.read()
            # If reading was successful, continue
            # processing. If not, try again
            if not ret:
                print('Problem connecting to camera!')
                continue
                
            # Exit process midway through by pressing 'q'
            if cv2.waitKey(10) == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return False

            # Try to detect face in the frame
            face_detected, face, rect = detect_face(frame)

            if face_detected: 
                # Display white rect around the face
                display_frame(frame, "Please look straight in the camera", True, rect, (255,255,255))

                # Try to recognize person in the photo
                face_recognized, person = recognize_face(face, known_emb)
                
                if face_recognized:
                    # Display green rect around the face
                    display_frame(frame, "Welcome, {}! Press ENTER to continue".format(person),
                                  True, rect, (0,255,0))
                    
                    # Wait for recognized person to press ENTER to gain access to the system
                    if cv2.waitKey(1) == 13:
                        # Release the camera
                        cap.release()
                        cv2.destroyAllWindows()
                        return True
                    
                # If face wasn't recognized
                else:
                    # After some time, if no face was recognized, stop execution
                    if time.time() - start > TIMEOUT:
                        print('Intruder alert! Access denied!')
                        # Display red rect around the face
                        display_frame(frame, "Intruder alert! Access denied!", True, rect, (0,0,255))
                        
                        # Wait for unrecognized person to press ENTER to exit
                        if cv2.waitKey(0) == 13:
                            # Release the camera
                            cap.release()
                            cv2.destroyAllWindows()
                            return False
                    # If timeout hasn't elapsed, try once again to recognize face
                    else:
                        continue

            # If no face was detected
            else:
                # Display what camera sees
                display_frame(frame, "Please look straight in the camera")
                # If q was pressed or time has expired, stop execution
                if time.time() - start > TIMEOUT:
                    print('No faces recognized! Access denied!')
                    # Release the camera
                    cap.release()
                    cv2.destroyAllWindows()
                    return False
    # Handle exception
    except Exception as e:
        print(e)
        cap.release()
        cv2.destroyAllWindows()
        return False

Try to gain access to the system

In [6]:
#access()

# Sleepiness detection

Constantly monitor face and calculate eye-aspect-ratio - a measure of how much the eyes are closed. If EAR goes below some empirical threshold, sound the alarm to wake driver up

In [ ]:
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import imutils
import time
import dlib
import cv2

# Define two constants, one for the eye aspect ratio to indicate
# closed eyes and a second constant for the number of seconds
# the EAR must be below the threshold to set off the alarm
EAR_THRESH = 0.28
EAR_TIMEOUT = 3

# Font for opencv
FONT = cv2.FONT_HERSHEY_SIMPLEX

# Path to wav file
ALARM_SOUND_PATH = 'sounds/alarm.wav'

# Initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
face_detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('pretrained_models/shape_predictor.dat')

# Grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]


def sound_alarm(path):
    """ Convenience function to play sound """
    playsound.playsound(path)
    
def calc_ear(eye):
    """
    Function to calculate eye aspect ratio
    
    Eye is passed as an np array of 6 elements containing [x, y] 
    coordinates of following points
                
                P1      P2
                 *      *
                   ---
                /       \  
    P0 *       (    O    )       * P3
                \       /
                   ---
                 *      *
                P5      P4
                 
    Arguments
    ---------
    eye - np array of 6 elements containing (x, y) coordinates of above
            described points on eye
    
    Returns
    -------
    EAR calculated as (dist(P1, P5) + dist(P2, P4))/(2 * dist(P0, P3))
    
    """
    # Compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])

    # Compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = np.linalg.norm(eye[0] - eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    return ear

def monitor_driver():
    """
    Wrapper function monitoring camera frames, detecting eyes and
    calculating EAR to determine if driver has dosed off, in which
    case it sounds an alarm. It goes on in a loop until 'q' is pressed
    """
    # Initialize the frame counter as well as a boolean used to
    # indicate if the alarm is going off
    ALARM_ON = False
    # Time point showing the exact moment driver fell asleep
    start = 0
    # Flag indicating if EAR has fallen bellow
    FLAG = 0
    
    try:
        # Instanciate capture object in order to
        # gain access to camera
        cap = cv2.VideoCapture(0)

        # Main loop looking for faces
        while True:
            # Capture a frame from camera
            ret, frame = cap.read()
            # If reading was successful, continue
            # processing. If not, try again
            if not ret:
                print('Problem connecting to camera!')
                continue

            # At any time pressing 'q' will close the program
            if cv2.waitKey(1) == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return
            
            # Turn image to bw
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale frame
            rects = face_detector(gray, 0)

            # Loop over the face detections, even though only one is expected
            for rect in rects:
                # Determine the facial landmarks for the face region, then
                # convert the facial landmark (x, y)-coordinates to a np array
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)

                # Extract the left and right eye coordinates, then use the
                # coordinates to compute the eye aspect ratio for both eyes
                left_eye = shape[lStart:lEnd]
                right_eye = shape[rStart:rEnd]
                left_ear = calc_ear(left_eye)
                right_ear = calc_ear(right_eye)
                
                # Average the eye aspect ratio together for both eyes
                EAR = (left_ear + right_ear) / 2.0

                # Compute the convex hull for the left and right eye, then
                # visualize each of the eyes
                left_eye_contour = cv2.convexHull(left_eye)
                right_eye_contour = cv2.convexHull(right_eye)
                cv2.drawContours(frame, [left_eye_contour], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [right_eye_contour], -1, (0, 255, 0), 1)

                # Check to see if the eye aspect ratio is below the 
                # threshold, and if so, increment the frame counter
                if EAR < EAR_THRESH:
                    if FLAG == 0:
                        FLAG = 1
                        start = time.time()

                    # If the eyes were closed for a sufficient number of frames
                    # then sound the alarm
                    if time.time() - start >= EAR_TIMEOUT:
                        # If the alarm is not on, turn it on
                        if not ALARM_ON:
                            ALARM_ON = True
                            # Start a thread to have the alarm
                            # sound played in the background
                            t = Thread(target=sound_alarm, args=(ALARM_SOUND_PATH,))
                            t.deamon = True
                            t.start()

                        # Print text to show drowsiness was detected
                        cv2.putText(frame, "WAKE UP!", (10, 30), FONT, 0.7, (0, 0, 0), 1)

                # otherwise, the eye aspect ratio is not below the blink
                # threshold, so reset the counter and alarm
                else:
                    FLAG = 0
                    ALARM_ON = False

                # Draw the computed eye aspect ratio on the frame to help
                # with debugging and setting the correct eye aspect ratio
                # thresholds and EAR_TIMEOUT
                cv2.putText(frame, "EAR: {:.2f}".format(EAR), (500, 30), FONT, 0.7, (0, 0, 0), 1)

            # show the frame
            cv2.imshow("Camera", frame)
                
    except Exception as e:
        print(e)
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
#monitor_driver()

# Run the complete program

In [ ]:
while True:
    print("\nMain menu:\n1. Add new person\n2. Monitor driver\n")

    option = int(input("Choose one option "))
    
    if option == 1:
        name = input('Enter persons name: ')
        take_pictures(name)
        create_embeddings(name)
        
    elif option == 2:
        if access():
            monitor_driver()
            
    else:
        break
        